## 8.4 多GPU计算

- 运行本节中的代码需要至少2块GPU；
- 事实上，一台机器上安装多块GPU很常见，这是因为主板上通常会有多个PCIe插槽。

In [1]:
!nvidia-smi

Sun Nov 14 13:12:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.43       Driver Version: 456.43       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  ERR!               WDDM  | 00000000:01:00.0 N/A |                  N/A |
| 30%   31C    P8    N/A /  N/A |    542MiB /  2048MiB |     N/A      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

- 大部分运算可以使用所有的CPU的全部计算资源，或者单块GPU的全部计算资源；
- 但如果使用多块GPU训练模型，我们需要实现相应的算法。这些算法中最常用的叫作数据并行。

### 8.4.1 数据并行

- 数据并行目前是深度学习里使用最广泛的将模型训练任务划分到多块GPU的方法。

- 假设一台机器上有$k$块GPU；
- 每块GPU及其相应的显存将分别独立维护一份完整的模型参数；
- 每块GPU将根据相应显存所分到的小批量子集和所维护的模型参数分别计算模型参数的本地梯度；
- 我们把$k$块显卡的显存上的本地梯度相加，便得到当前的小批量随机梯度。
- 之后，每块GPU都使用这个小批量随机梯度分别更新相应显存所维护的那一份完整的模型参数

In [3]:
import d2lzh as d2l
import mxnet as mx
from mxnet import autograd, nd
from mxnet.gluon import loss as gloss
import time

### 8.4.2 定义模型

In [4]:
# 初始化模型参数
scale = 0.01
w1 = nd.random_normal(scale=scale, shape=(20, 1, 3, 3))
b1 = nd.zeros(shape=20)
w2 = nd.random_normal(scale=scale, shape=(50, 20, 5, 5))
b2 = nd.zeros(shape=50)
w3 = nd.random_normal(scale=scale, shape=(800, 128))
b3 = nd.zeros(shape=128)
w4 = nd.random_normal(scale=scale, shape=(128,10))
b4 = nd.zeros(shape=10)
params = [w1, b1, w2, b2, w3, b3, w4, b4]

# 定义模型
def lenet(X, params):
    h1_conv = nd.Convolution(data=X, weight=params[0], bias=params[1],
                             kernel=(3, 3), num_filter=20)
    h1_activation = nd.relu(h1_conv)
    h1 = nd.Pooling(data=h1_activation, pool_type='avg', kernel=(2, 2),
                    stride=(2, 2))
    h2_conv = nd.Convolution(data=h1, weight=params[2], bias=[3],
                             kernel=(5, 5), num_filter=50)
    h2_activation = nd.relu(h2_conv)
    h2 = nd.Pooling(data=h2_activation, pool_type='avg', kernel=(2, 2))
    h3_linear = nd.dot(h2, params[4]) + params[5]
    h3 = nd.relu(h3_linear)
    y_hat = nd.dot(h3, params[6]) + params[7]
    return y_hat

# 交叉熵损失函数
loss = gloss.SoftmaxCrossEntropyLoss()

### 8.4.3 多GPU之间同步数据

下面的get_params函数将模型参数复制到某块显卡的显存并初始化梯度。

In [5]:
def get_params(params, ctx):
    new_params = [p.copyto(ctx) for p in params]
    for p in new_params:
        p.attach_grad()
    return new_params

In [7]:
new_params = get_params(params, mx.gpu(0))
print('b1 weight:', new_params[1])
print('b1 grad:', new_params[1].grad)

MXNetError: [17:00:51] C:\Jenkins\workspace\mxnet-tag\mxnet\src\ndarray\ndarray.cc:1285: GPU is not enabled

- 下面的allreduce函数可以把各块显卡的显存上的数据加起来，然后再广播到所有的显存上。|

In [8]:
def allreduce(data):
    for i in range(1, len(data)):
        data[0][:] += data[i].copyto(data[0].context)
    for i in range(1, len(data)):
        data[0].copyto(data[i])

In [9]:
data = [nd.ones((1, 2), ctx=mx.gpu(i)) * (i + 1) for i in range(2)]
print('before allreduce:', data)
allreduce(data)
print('after allreduce:', data)

MXNetError: [19:30:07] C:\Jenkins\workspace\mxnet-tag\mxnet\src\imperative\imperative.cc:81: Operator _ones is not implemented for GPU.

- 给定一个批量的数据样本，下面的split_and_load函数可以将其划分并复制到各块显卡的显存上。

In [10]:
def split_and_load(data, ctx):
    n, k = data.shape[0], len(ctx)
    m = n // k # 简单起见，假设可以整除
    assert m * k == n, '# examples is not divided by # devices.'
    return [data[i * m: (i+1) * m].as_in_context(ctx[i]) for i in range(k)]

In [11]:
batch = nd.arange(24).reshape((6, 4))
ctx = [mx.gpu(0), mx.gpu(1)]
splitted = split_and_load(batch, ctx)
print('input:', batch)
print('load into:', ctx)
print('output:', splitted)

MXNetError: [19:48:48] C:\Jenkins\workspace\mxnet-tag\mxnet\src\ndarray\ndarray.cc:1285: GPU is not enabled

### 8.4.4 单个小批量上的多GPU训练

In [12]:
def train_batch(X, y, gpu_params, ctx, lr):
    # 当ctx包含多块gpu及相应的显存时，将小批量数据样本划分并复制到各个显存上
    gpu_Xs, gpu_ys = split_and_load(X, ctx), split_and_load(y, ctx)
    with autograd.record(): # 在各块gpu上分别计算损失
        ls = [loss(lenet(gpu_X, gpu_W), gpu_y)
              for gpu_X,gpu_y, gpu_W in zip(gpu_Xs, gpu_ys, gpu_params)]
        for l in ls: # 在各块GPU上分别反向传播
            l.backward()
        # 把各块显卡的显存上的梯度加起来，然后广播到所有显存上
        for i in range(len(gpu_params[0])):
            allreduce([gpu_params[c][i].grad for c in range(len(ctx))])
        for param in gpu_params: # 在各块显卡的显存上分别更新模型参数
            d2l.sgd(param, lr, X.shape[0]) # 这里使用了完整批量大小

### 8.4.5 定义训练函数

- 值得强调的是，在这里我们需要依据数据并行将完整的模型参数复制到多块显卡的显存上，并在每次迭代时对单个小批量进行多GPU训练。

In [13]:
def train(num_gpus, batch_size, lr):
    train_iter,test_iter = d2l.load_data_fashion_mnist(batch_size)
    ctx = [mx.gpu(i) for i in range(num_gpus)]
    print('running on:', ctx)
    # 将模型参数复制到num_gpus块显卡的显存上
    gpu_params = [get_params(params, c) for c in ctx]
    for epoch in range(4):
        start = time.time()
        for X, y in train_iter:
            # 对单个小批量进行多GPU训练
            train_batch(X, y, gpu_params, ctx, lr)
            nd.waitall()
        train_time = time.time() - start
        
        def net(x): # 在gpu(0)上验证模型
            return lenet(x, gpu_params[0])
        
        test_acc = d2l.evaluate_accuracy(test_iter, net, ctx[0])
        print('epoch %d, time %.1f sec, test acc %.2f'
              % (epoch + 1, train_time, test_acc))

### 8.4.6 多GPU训练实验

In [14]:
train(num_gpus=1, batch_size=256, lr=0.2)

running on: [gpu(0)]


MXNetError: [21:46:50] C:\Jenkins\workspace\mxnet-tag\mxnet\src\ndarray\ndarray.cc:1285: GPU is not enabled

In [15]:
train(num_gpus=2, batch_size=256, lr=0.2)

running on: [gpu(0), gpu(1)]


MXNetError: [21:51:31] C:\Jenkins\workspace\mxnet-tag\mxnet\src\ndarray\ndarray.cc:1285: GPU is not enabled

- 因为有额外的通信开销，所以我们并没有看到训练时间的显著降低。

# 小结

- 可以使用数据并行更充分的利用多块GPU的计算资源，实现多GPU训练模型。
- 给定超参数的情况下，改变GPU数量时模型的训练准确率大体相当。